#### Importing required Libraries

In [1]:
import numpy as np 
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library
print('Libraries imported.')

Libraries imported.


 #### Load and explore the data

In [2]:
with open('nyu-2451-34572-geojson.json') as json_data:
    newyork_data = json.load(json_data)

In [3]:
neighborhoods_data = newyork_data['features']

In [4]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

#### Tranforming the data into a *pandas* dataframe

In [5]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighborhoods = pd.DataFrame(columns=column_names)

In [6]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [7]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [8]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [9]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(len(neighborhoods['Borough'].unique()),
                                                                   neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


#### Using geopy library to get the latitude and longitude values of New York City

In [10]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


#### Create a map of New York with neighborhoods superimposed on top.

In [11]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

### Brooklyn data

In [12]:
Brooklyn_data = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)
Brooklyn_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


In [13]:
address = 'Brooklyn, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Brooklyn are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Brooklyn are 40.6501038, -73.9495823.


In [14]:
map_brooklyn = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Brooklyn_data['Latitude'], Brooklyn_data['Longitude'],Brooklyn_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_brooklyn)  
    
map_brooklyn

#### Define Foursquare Credentials and Version

HIDDEN

#### Exploring the first neighborhood in Manhattan

In [16]:
Brooklyn_data.loc[0, 'Neighborhood']

'Bay Ridge'

In [17]:
neighborhood_latitude = Brooklyn_data.loc[0, 'Latitude'] 
neighborhood_longitude = Brooklyn_data.loc[0, 'Longitude']
neighborhood_name = Brooklyn_data.loc[0, 'Neighborhood'] 
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, neighborhood_latitude, neighborhood_longitude))

Latitude and longitude values of Bay Ridge are 40.625801065010656, -74.03062069353813.


In [18]:
# type your answer here
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url



'https://api.foursquare.com/v2/venues/explore?&client_id=DW151YWKST1M250SCVZCM2YG4CA3KBBGOWT5ZKYORNZ25ZAR&client_secret=X0JXLOLKEW01XMGJMJ5VTAPJD5ZZMOGLMJKHLLHZWJ3XFEWY&v=20180604&ll=40.625801065010656,-74.03062069353813&radius=500&limit=100'

#### Getting the top 100 venues that are in Marble Hill within a radius of 500 meters.

In [19]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d74ebc5bbed210038669ce8'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bay Ridge',
  'headerFullLocation': 'Bay Ridge, Brooklyn',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 89,
  'suggestedBounds': {'ne': {'lat': 40.63030106951066,
    'lng': -74.02470273356597},
   'sw': {'lat': 40.62130106051065, 'lng': -74.03653865351028}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b895827f964a5206c2d32e3',
       'name': 'Pilo Arts Day Spa and Salon',
       'location': {'address': '8412 3rd Ave',
        'lat': 40.62474788273414,
        'lng': -74.03059056940135,
        'labeledLatL

In [20]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [21]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Pilo Arts Day Spa and Salon,Spa,40.624748,-74.030591
1,Bagel Boy,Bagel Shop,40.627896,-74.029335
2,Cocoa Grinder,Juice Bar,40.623967,-74.030863
3,Pegasus Cafe,Breakfast Spot,40.623168,-74.031186
4,Ho' Brah Taco Joint,Taco Place,40.622960,-74.031371


In [22]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

89 venues were returned by Foursquare.


In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Creating a dataframe of all the venues of all the neighbourhoods in Brooklyn

In [24]:
# type your answer here

brooklyn_venues = getNearbyVenues(names=Brooklyn_data['Neighborhood'],
                                   latitudes=Brooklyn_data['Latitude'],longitudes=Brooklyn_data['Longitude'])

Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker Heights
Gerritsen Beach
Marine Park
Clinton Hill
Sea Gate
Downtown
Boerum Hill
Prospect Lefferts Gardens
Ocean Hill
City Line
Bergen Beach
Midwood
Prospect Park South
Georgetown
East Williamsburg
North Side
South Side
Ocean Parkway
Fort Hamilton
Ditmas Park
Wingate
Rugby
Remsen Village
New Lots
Paerdegat Basin
Mill Basin
Fulton Ferry
Vinegar Hill
Weeksville
Broadway Junction
Dumbo
Homecrest
Highland Park
Madison
Erasmus


In [25]:
brooklyn_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bay Ridge,40.625801,-74.030621,Pilo Arts Day Spa and Salon,40.624748,-74.030591,Spa
1,Bay Ridge,40.625801,-74.030621,Bagel Boy,40.627896,-74.029335,Bagel Shop
2,Bay Ridge,40.625801,-74.030621,Cocoa Grinder,40.623967,-74.030863,Juice Bar
3,Bay Ridge,40.625801,-74.030621,Pegasus Cafe,40.623168,-74.031186,Breakfast Spot
4,Bay Ridge,40.625801,-74.030621,Ho' Brah Taco Joint,40.622960,-74.031371,Taco Place


In [26]:
print(brooklyn_venues.shape)
brooklyn_venues.head()

(2849, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bay Ridge,40.625801,-74.030621,Pilo Arts Day Spa and Salon,40.624748,-74.030591,Spa
1,Bay Ridge,40.625801,-74.030621,Bagel Boy,40.627896,-74.029335,Bagel Shop
2,Bay Ridge,40.625801,-74.030621,Cocoa Grinder,40.623967,-74.030863,Juice Bar
3,Bay Ridge,40.625801,-74.030621,Pegasus Cafe,40.623168,-74.031186,Breakfast Spot
4,Bay Ridge,40.625801,-74.030621,Ho' Brah Taco Joint,40.622960,-74.031371,Taco Place


#### Let's find out how many unique categories can be curated from all the returned venues

In [27]:
print('There are {} uniques categories.'.format(len(brooklyn_venues['Venue Category'].unique())))

There are 287 uniques categories.


In [28]:
# one hot encoding
brooklyn_onehot = pd.get_dummies(brooklyn_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
brooklyn_onehot['Neighborhood'] = brooklyn_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [brooklyn_onehot.columns[-1]] + list(brooklyn_onehot.columns[:-1])
brooklyn_onehot = brooklyn_onehot[fixed_columns]

brooklyn_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Airport Terminal,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,History Museum,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Road,Rock Club,Roller Rink,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Ski Area,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Tiki Bar,Toy / Game Store,Trail,Turkish Restaurant,Used Bookstore,Vape Store,

In [29]:
brooklyn_grouped = brooklyn_onehot.groupby('Neighborhood').mean().reset_index()
brooklyn_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Airport Terminal,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,History Museum,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Non-Profit,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Road,Rock Club,Roller Rink,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Ski Area,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Tiki Bar,Toy / Game Store,Trail,Turkish Restaurant,Used Bookstore,Vape Store,

In [30]:
brooklyn_grouped.shape

(70, 287)

#### Printing each neighborhood along with the top 10 most common venues

In [31]:
num_top_venues = 10

for hood in brooklyn_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = brooklyn_grouped[brooklyn_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bath Beach----
                  venue  freq
0    Chinese Restaurant  0.06
1              Pharmacy  0.06
2       Bubble Tea Shop  0.04
3            Kids Store  0.04
4  Fast Food Restaurant  0.04
5           Pizza Place  0.04
6            Donut Shop  0.04
7    Italian Restaurant  0.04
8      Sushi Restaurant  0.04
9  Cantonese Restaurant  0.02


----Bay Ridge----
                       venue  freq
0                        Spa  0.07
1         Italian Restaurant  0.07
2                Pizza Place  0.04
3           Greek Restaurant  0.03
4        American Restaurant  0.03
5                        Bar  0.03
6                 Bagel Shop  0.03
7  Middle Eastern Restaurant  0.02
8         Chinese Restaurant  0.02
9         Seafood Restaurant  0.02


----Bedford Stuyvesant----
            venue  freq
0            Café  0.07
1     Pizza Place  0.07
2             Bar  0.07
3     Coffee Shop  0.07
4        Wine Bar  0.04
5      Bagel Shop  0.04
6        Boutique  0.04
7  Sandwich Place  0.04
8

                  venue  freq
0         Deli / Bodega  0.25
1          Liquor Store  0.12
2    Chinese Restaurant  0.12
3  Fast Food Restaurant  0.12
4         Event Service  0.06
5         Metro Station  0.06
6              Pharmacy  0.06
7                   Gym  0.06
8           Pizza Place  0.06
9           Bus Station  0.06


----East Williamsburg----
                           venue  freq
0                            Bar  0.08
1                   Cocktail Bar  0.06
2                    Coffee Shop  0.06
3                         Bakery  0.06
4                  Deli / Bodega  0.06
5                   Concert Hall  0.05
6             Mexican Restaurant  0.05
7                 Sandwich Place  0.03
8           Gym / Fitness Center  0.03
9  Vegetarian / Vegan Restaurant  0.03


----Erasmus----
                  venue  freq
0  Caribbean Restaurant  0.21
1           Yoga Studio  0.05
2         Grocery Store  0.05
3           Music Venue  0.05
4                   Bar  0.05
5              

                     venue  freq
0            Moving Target  0.17
1                     Food  0.17
2       Chinese Restaurant  0.17
3         Asian Restaurant  0.17
4             Home Service  0.17
5                 Bus Line  0.17
6  New American Restaurant  0.00
7                Nightclub  0.00
8               Non-Profit  0.00
9               Nail Salon  0.00


----Park Slope----
                    venue  freq
0             Coffee Shop  0.06
1            Burger Joint  0.06
2     American Restaurant  0.06
3      Italian Restaurant  0.04
4             Pizza Place  0.04
5                     Spa  0.03
6                  Bakery  0.03
7              Bagel Shop  0.03
8  Furniture / Home Store  0.03
9               Pet Store  0.03


----Prospect Heights----
                venue  freq
0                 Bar  0.07
1  Mexican Restaurant  0.05
2              Bakery  0.04
3           Wine Shop  0.04
4        Cocktail Bar  0.04
5        Gourmet Shop  0.04
6      Ice Cream Shop  0.04
7     Thai Re

In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Creating a new dataframe and display the top 10 venues for each neighborhood.

In [33]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = brooklyn_grouped['Neighborhood']

for ind in np.arange(brooklyn_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(brooklyn_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bath Beach,Chinese Restaurant,Pharmacy,Pizza Place,Kids Store,Italian Restaurant,Bubble Tea Shop,Fast Food Restaurant,Sushi Restaurant,Donut Shop,Burger Joint
1,Bay Ridge,Spa,Italian Restaurant,Pizza Place,Bagel Shop,Greek Restaurant,American Restaurant,Bar,Ice Cream Shop,Sandwich Place,Chinese Restaurant
2,Bedford Stuyvesant,Coffee Shop,Café,Pizza Place,Bar,Juice Bar,New American Restaurant,Cocktail Bar,Fruit & Vegetable Store,Gourmet Shop,Bus Stop
3,Bensonhurst,Chinese Restaurant,Ice Cream Shop,Italian Restaurant,Donut Shop,Grocery Store,Sushi Restaurant,Spa,Food Truck,Pet Store,Smoke Shop
4,Bergen Beach,Harbor / Marina,Donut Shop,Baseball Field,Playground,Athletics & Sports,Women's Store,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant


In [34]:
print('There are {} uniques categories.'.format(len(brooklyn_venues['Venue Category'].unique())))

print('Here is the list of different categories:')
list(brooklyn_venues['Venue Category'].unique())

There are 287 uniques categories.
Here is the list of different categories:


['Spa',
 'Bagel Shop',
 'Juice Bar',
 'Breakfast Spot',
 'Taco Place',
 'Grocery Store',
 'Caucasian Restaurant',
 'Bookstore',
 'Middle Eastern Restaurant',
 'Hookah Bar',
 'Coffee Shop',
 'Sports Bar',
 'Bar',
 'Italian Restaurant',
 'Greek Restaurant',
 'Pizza Place',
 'Mexican Restaurant',
 'Ice Cream Shop',
 'Tea Room',
 'Lounge',
 'Optical Shop',
 'American Restaurant',
 'Chinese Restaurant',
 'Sushi Restaurant',
 'New American Restaurant',
 'Department Store',
 'Vietnamese Restaurant',
 'Thai Restaurant',
 'Clothing Store',
 'Seafood Restaurant',
 'Toy / Game Store',
 'Donut Shop',
 'Video Game Store',
 'Indian Restaurant',
 'Lingerie Store',
 'Sandwich Place',
 'Pool Hall',
 'Pharmacy',
 'Dim Sum Restaurant',
 'Snack Place',
 'Mobile Phone Shop',
 'Kids Store',
 'Fast Food Restaurant',
 'Hotel',
 'Gym / Fitness Center',
 'Mattress Store',
 'Supermarket',
 'Diner',
 'Art Gallery',
 'Cosmetics Shop',
 'Hardware Store',
 'Paper / Office Supplies Store',
 'Deli / Bodega',
 'Bakery'

In [35]:
# This list is created manually 
important_list_of_features = [
 
 'Neighborhood',
 'Juice Bar',
 'Breakfast Spot',
 'Taco Place',
 
 'Caucasian Restaurant',
 
 'Middle Eastern Restaurant',
 
 'Coffee Shop',
 
 'Italian Restaurant',
 'Greek Restaurant',
 'Pizza Place',
 'Mexican Restaurant',
 'Ice Cream Shop',
 
 
 'American Restaurant',
 'Chinese Restaurant',
 'Sushi Restaurant',
 'New American Restaurant',

 'Vietnamese Restaurant',
 'Thai Restaurant',
 
 'Seafood Restaurant',
 
 'Indian Restaurant',
 
 'Sandwich Place',
 
 'Dim Sum Restaurant',
 'Snack Place',
 
 'Fast Food Restaurant',
 
 'Deli / Bodega',
 'Bakery',
 
 'Shabu-Shabu Restaurant',
 'Noodle House',
 
 'Hotpot Restaurant',
 'Russian Restaurant',
 'Dumpling Restaurant',
 
 'Food Truck',
 'Latin American Restaurant',
 
 'Polish Restaurant',
 
 'Café',
 
 'French Restaurant',
 
 'Restaurant',
 'Vegetarian / Vegan Restaurant',
 
 'Fried Chicken Joint',
 
 'Falafel Restaurant',
 
 'Ramen Restaurant',
 'Mediterranean Restaurant',
 'Eastern European Restaurant',
 'Korean Restaurant',
 'Varenyky restaurant',
 'Turkish Restaurant',
 'Buffet',
 'Steakhouse',
 'Japanese Restaurant',
 'Caribbean Restaurant',
 'Burger Joint',
 'Salad Place',
 'Cajun / Creole Restaurant',
 'Southern / Soul Food Restaurant',
 'Spanish Restaurant',
 'Tapas Restaurant',
 'Cuban Restaurant',
 'Pakistani Restaurant',
 'Food Stand',
 'BBQ Joint',
 'Asian Restaurant',
 'Burrito Place',
 'Ethiopian Restaurant',
 'Argentinian Restaurant',
 'Filipino Restaurant',
 'Wings Joint',
 'Israeli Restaurant',
 'German Restaurant',
 'Food Court',
 'South American Restaurant',
 'Peruvian Restaurant',
 'Cantonese Restaurant',
 'Kebab Restaurant',
 'Shanghai Restaurant',
 'Hawaiian Restaurant',
 'Halal Restaurant',
 'Taiwanese Restaurant',
 'Lebanese Restaurant',
 'Arepa Restaurant',
 'Jewish Restaurant',
 'Tibetan Restaurant',]

In [36]:
brooklyn_onehot = brooklyn_onehot[important_list_of_features].groupby(
    'Neighborhood').sum()


brooklyn_onehot.head()

,Juice Bar,Breakfast Spot,Taco Place,Caucasian Restaurant,Middle Eastern Restaurant,Coffee Shop,Italian Restaurant,Greek Restaurant,Pizza Place,Mexican Restaurant,Ice Cream Shop,American Restaurant,Chinese Restaurant,Sushi Restaurant,New American Restaurant,Vietnamese Restaurant,Thai Restaurant,Seafood Restaurant,Indian Restaurant,Sandwich Place,Dim Sum Restaurant,Snack Place,Fast Food Restaurant,Deli / Bodega,Bakery,Shabu-Shabu Restaurant,Noodle House,Hotpot Restaurant,Russian Restaurant,Dumpling Restaurant,Food Truck,Latin American Restaurant,Polish Restaurant,Café,French Restaurant,Restaurant,Vegetarian / Vegan Restaurant,Fried Chicken Joint,Falafel Restaurant,Ramen Restaurant,Mediterranean Restaurant,Eastern European Restaurant,Korean Restaurant,Varenyky restaurant,Turkish Restaurant,Buffet,Steakhouse,Japanese Restaurant,Caribbean Restaurant,Burger Joint,Salad Place,Cajun / Creole Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Tapas Restaurant,Cuban Restaurant,Pakistani Restaurant,Food Stand,BBQ Joint,Asian Restaurant,Burrito Place,Ethiopian Restaurant,Argentinian Restaurant,Filipino Restaurant,Wings Joint,Israeli Restaurant,German Restaurant,Food Court,South American Restaurant,Peruvian Restaurant,Cantonese Restaurant,Kebab Restaurant,Shanghai Restaurant,Hawaiian Restaurant,Halal Restaurant,Taiwanese Restaurant,Lebanese Restaurant,Arepa Restaurant,Jewish Restaurant,Tibetan Restaurant
Neighborhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Bath Beach,0,0,0,0,0,1,2,0,2,0,1,0,3,2,0,0,0,0,0,0,0,0,2,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,1,1,0,0,0,0,0,0,0,0
Bay Ridge,1,1,1,1,2,1,6,3,4,1,2,3,2,2,1,1,2,2,1,2,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Bedford Stuyvesant,1,0,0,0,0,2,0,0,2,0,0,0,0,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,2,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Bensonhurst,0,0,0,0,0,1,2,0,1,0,2,0,4,2,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Bergen Beach,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [37]:
featurename_list = list(brooklyn_onehot.columns)
restaurant_list = []
for counter, value in enumerate(featurename_list):
    if value.find('Restaurant') != (-1):
        restaurant_list.append(value)
        
brooklyn_onehot['Total Restaurants'] = brooklyn_onehot[restaurant_list].sum(axis = 1)
brooklyn_onehot = brooklyn_onehot.drop(columns = restaurant_list)
feat_name_list = list(brooklyn_onehot.columns)
joint_list = []
for counter, value in enumerate(featurename_list):
    if value.find('Joint') != (-1):
        joint_list.append(value)
        
brooklyn_onehot['Total Joints'] = brooklyn_onehot[joint_list].sum(axis = 1)
brooklyn_onehot = brooklyn_onehot.drop(columns = joint_list)

In [38]:
brooklyn_onehot

,Juice Bar,Breakfast Spot,Taco Place,Coffee Shop,Pizza Place,Ice Cream Shop,Sandwich Place,Snack Place,Deli / Bodega,Bakery,Noodle House,Food Truck,Café,Varenyky restaurant,Buffet,Steakhouse,Salad Place,Food Stand,Burrito Place,Food Court,Total Restaurants,Total Joints
Neighborhood,,,,,,,,,,,,,,,,,,,,,,
Bath Beach,0,0,0,1,2,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,18,1
Bay Ridge,1,1,1,1,4,2,2,1,1,1,0,0,0,0,0,0,0,0,0,0,29,0
Bedford Stuyvesant,1,0,0,2,2,0,1,0,1,0,0,0,2,0,0,0,0,0,0,0,3,2
Bensonhurst,0,0,0,1,1,2,0,0,0,1,1,1,0,0,0,0,0,0,0,0,12,0
Bergen Beach,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Boerum Hill,1,0,0,4,0,0,3,0,2,2,0,1,0,0,0,0,0,0,1,1,17,1
Borough Park,0,0,0,1,3,0,0,0,1,1,0,0,2,0,0,0,0,0,0,0,4,0
Brighton Beach,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,15,0
Broadway Junction,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,2,2


In [39]:
# import k-means from clustering stage
from sklearn.cluster import KMeans
# run k-means clustering
kmeans = KMeans(n_clusters = 5, random_state = 0).fit(brooklyn_onehot)

In [40]:
means_df = pd.DataFrame(kmeans.cluster_centers_)
means_df.columns = brooklyn_onehot.columns
means_df.index = ['G1','G2','G3','G4','G5']
means_df['Total Sum'] = means_df.sum(axis = 1)
means_df.sort_values(axis = 0, by = ['Total Sum'], ascending=False)

,Juice Bar,Breakfast Spot,Taco Place,Coffee Shop,Pizza Place,Ice Cream Shop,Sandwich Place,Snack Place,Deli / Bodega,Bakery,Noodle House,Food Truck,Café,Varenyky restaurant,Buffet,Steakhouse,Salad Place,Food Stand,Burrito Place,Food Court,Total Restaurants,Total Joints,Total Sum
G4,0.600000,8.000000e-01,4.000000e-01,2.200000,3.800000,0.800000,1.800000,2.000000e-01,0.800000,1.000000,0.000000e+00,0.000000,0.400000,0.000000e+00,0.000000e+00,2.000000e-01,0.000000e+00,0.000000e+00,2.000000e-01,4.000000e-01,30.200000,2.200000,46.000000
G2,0.750000,3.750000e-01,1.250000e-01,4.250000,3.750000,1.750000,0.750000,-1.734723e-18,2.250000,2.625000,-1.734723e-18,0.250000,1.000000,-1.734723e-18,-1.734723e-18,1.387779e-17,2.500000e-01,1.250000e-01,1.250000e-01,1.387779e-17,23.500000,1.375000,43.250000
G3,0.307692,7.692308e-02,3.076923e-01,1.615385,1.769231,1.000000,1.000000,-3.469447e-18,1.307692,1.538462,7.692308e-02,0.538462,1.384615,7.692308e-02,-3.469447e-18,2.307692e-01,1.538462e-01,-3.469447e-18,7.692308e-02,3.076923e-01,13.153846,0.846154,25.769231
G1,0.181818,3.181818e-01,4.545455e-02,0.636364,1.500000,0.409091,0.500000,-3.469447e-18,0.909091,0.500000,-3.469447e-18,0.136364,0.454545,-3.469447e-18,4.545455e-02,4.545455e-02,1.363636e-01,-3.469447e-18,1.387779e-17,-2.775558e-17,5.681818,0.909091,12.409091
G5,0.045455,-5.551115e-17,-2.775558e-17,0.227273,0.818182,0.363636,0.090909,-3.469447e-18,0.545455,0.227273,-3.469447e-18,0.136364,0.409091,-3.469447e-18,-3.469447e-18,9.090909e-02,-2.775558e-17,-3.469447e-18,1.387779e-17,4.545455e-02,1.363636,0.363636,4.727273


# Result:


In [41]:
neigh = pd.DataFrame([brooklyn_onehot.index,1 + kmeans.labels_]).T
neigh.columns = ['Neighborhood', 'Group']
neigh.head()

,Neighborhood,Group
0,Bath Beach,3
1,Bay Ridge,4
2,Bedford Stuyvesant,1
3,Bensonhurst,3
4,Bergen Beach,5


### Best Group is G4
### Second Best Group is G2
### Third Best Group is G3
### Fourth Best Group is G1

# Conclusion:
## Best Neighborhoods to open a market are

In [42]:
neigh[neigh['Group'] == 4]

,Neighborhood,Group
1,Bay Ridge,4
15,Clinton Hill,4
21,Downtown,4
30,Fort Greene,4
62,South Side,4


## Second Best Neighborhoods are

In [44]:
neigh[neigh['Group'] == 2]

,Neighborhood,Group
9,Brooklyn Heights,2
11,Bushwick,2
13,Carroll Gardens,2
16,Cobble Hill,2
37,Greenpoint,2
49,North Side,2
53,Park Slope,2
54,Prospect Heights,2


## Third Best Neighborhoods are

In [45]:
neigh[neigh['Group'] == 3]

,Neighborhood,Group
0,Bath Beach,3
3,Bensonhurst,3
5,Boerum Hill,3
7,Brighton Beach,3
20,Ditmas Park,3
22,Dumbo,3
26,East Williamsburg,3
31,Fort Hamilton,3
32,Fulton Ferry,3
35,Gowanus,3


## Fourth Best Neighborhoods are

In [46]:
neigh[neigh['Group'] == 1]

,Neighborhood,Group
2,Bedford Stuyvesant,1
6,Borough Park,1
10,Brownsville,1
14,City Line,1
19,Cypress Hills,1
25,East New York,1
27,Erasmus,1
28,Flatbush,1
29,Flatlands,1
33,Georgetown,1
